In [ ]:
import pandas as pd
import numpy as np 


In [14]:
#import QB stats CSV
DF = pd.read_csv('supp/2020QB.csv')
#splitting on '\' to keep player name
Name = DF['Player'].str.split('\\').str[0]
Name = Name.values.tolist()
#function to remove puncutaion from player names
def remove_punc(string):
    punc = '*+'
    for ele in string:
        if ele in punc:
            string = string.replace(ele,'')
    return string
Name = [remove_punc(i) for i in Name]
DF['Player Name'] = Name
DF = DF.iloc[:, 2:]
#creating DF with fields I want
DF_QB = DF[['Player Name', 'Age', 'Pos', 'Tm', 'G', 'GS', 'Cmp', 'Att', 'Yds', 'TD']]
DF_QB.head(10)

,Player Name,Age,Pos,Tm,G,GS,Cmp,Att,Yds,TD
0,Matt Ryan,35,QB,ATL,16,16,407,626,4581,26
1,Tom Brady,43,QB,TAM,16,16,401,610,4633,40
2,Ben Roethlisberger,38,QB,PIT,15,15,399,608,3803,33
3,Justin Herbert,22,QB,LAC,15,15,396,595,4336,31
4,Patrick Mahomes,25,QB,KAN,15,15,390,588,4740,38
5,Josh Allen,24,QB,BUF,16,16,396,572,4544,37
6,Kyler Murray,23,QB,ARI,16,16,375,558,3971,26
7,Russell Wilson,32,QB,SEA,16,16,384,558,4212,40
8,Jared Goff,26,QB,LAR,15,15,370,552,3952,20
9,Deshaun Watson,25,QB,HOU,16,16,382,544,4823,33


In [20]:
DFPM = pd.read_csv('supp/playermeasureables - Sheet1.csv')
Last_NamePM = DFPM['Player'].str.split(', ').str[0]
Last_NamePM = Last_NamePM.values.tolist()
First_NamePM = DFPM['Player'].str.split(', ').str[1]
First_NamePM = First_NamePM.values.tolist()
DFPM['First Name'] = First_NamePM
DFPM['Last Name'] = Last_NamePM
DFPM['Player Name'] = DFPM[['First Name', 'Last Name']].apply(lambda x: ' '.join(x), axis=1)
DFPM_Info = DFPM[['Player Name', 'Pos', 'Team', 'Height', 'Weight', 'DOB']]
DFPM_Info.head(10)


,Player Name,Pos,Team,Height,Weight,DOB
0,Ameer Abdullah,RB,MIN,"5'9""",203 lbs,6/13/93
1,Oday Aboushi,G,DET,"6'5""",315 lbs,6/5/91
2,Johnathan Abram,S,LV,"6'0""",205 lbs,10/25/96
3,Delrick Abrams,DB,ATL,"6'3""",180 lbs,4/4/97
4,Andrew Adams,S,TB,"5'11""",205 lbs,10/28/92
5,Davante Adams,WR,GB,"6'1""",215 lbs,12/24/92
6,Jamal Adams,S,SEA,"6'1""",213 lbs,10/17/95
7,Josh Adams,RB,NYJ,"6'2""",225 lbs,10/26/96
8,Matthew Adams,LB,IND,"6'0""",229 lbs,12/12/95
9,Montravius Adams,DL,GB,"6'4""",304 lbs,7/24/95
